###Fuente de la data

Los micro-datos gestionados en este notebook son tomados de www.datos.gov.co y corresponden a datos cuya fuente primaria es el Ministerio de Defensa Nacional de Colombia

In [10]:
import pandas as pd

###Archivo 26zg-9p9r.csv

Contiene información sobre incautaciones cocaína.  Incluye el número de kilos de clorhidrato de cocaína incautado o interrumpido por la Fuerza Pública en el ejercicio de sus funciones. Entiéndase por Clorhidrato de cocaína como el producto obtenido a partir de la pasta básica de cocaína o base de cocaína mediante una serie de reacciones que incluyen cambios de pH y procesos de precipitación, para finalizar con la adición de ácido clorhídrico con el objeto de formar la sal. La unidad de medida corresponde a kilogramos incautados (kg).

In [11]:
df = pd.read_csv("https://www.datos.gov.co/resource/26zg-9p9r.csv?$limit=500000", dtype={'cod_muni': str})
df.head()

<ipython-input-11-c83d3d53fa28>:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("https://www.datos.gov.co/resource/26zg-9p9r.csv?$limit=500000", dtype={'cod_muni': str})


,fecha_hecho,cod_depto,departamento,cod_muni,municipio,cantidad,unidad
0,2024-07-31T00:00:00.000,05,ANTIOQUIA,05040,ANORI,0.0100,KILOGRAMO
1,2024-07-31T00:00:00.000,05,ANTIOQUIA,05148,EL CARMEN DE VIBORAL,0.0172,KILOGRAMO
2,2024-07-31T00:00:00.000,05,ANTIOQUIA,05001,MEDELLIN,0.0250,KILOGRAMO
3,2024-07-31T00:00:00.000,05,ANTIOQUIA,05001,MEDELLIN,0.0970,KILOGRAMO
4,2024-07-31T00:00:00.000,05,ANTIOQUIA,05001,MEDELLIN,0.0052,KILOGRAMO


##Revisión y limpieza para integrarlo a la base de datos

### - Resumen de la estructura del dataset

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197074 entries, 0 to 197073
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   fecha_hecho   197074 non-null  object 
 1   cod_depto     197038 non-null  object 
 2   departamento  197074 non-null  object 
 3   cod_muni      197025 non-null  object 
 4   municipio     197074 non-null  object 
 5   cantidad      197074 non-null  float64
 6   unidad        197074 non-null  object 
dtypes: float64(1), object(6)
memory usage: 10.5+ MB


### Eliminación de columnas irrelevantes para el proyecto

In [13]:
relevant_cols = ['cod_muni', 'departamento', 'municipio', 'fecha_hecho', 'cantidad', 'unidad']
cocaine_confiscations_MinDefensa = df[relevant_cols]
cocaine_confiscations_MinDefensa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197074 entries, 0 to 197073
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   cod_muni      197025 non-null  object 
 1   departamento  197074 non-null  object 
 2   municipio     197074 non-null  object 
 3   fecha_hecho   197074 non-null  object 
 4   cantidad      197074 non-null  float64
 5   unidad        197074 non-null  object 
dtypes: float64(1), object(5)
memory usage: 9.0+ MB


### Convertir columna fecha_hecho a tipo date

In [14]:
# Hacer una copia explícita del DataFrame
cocaine_confiscations_MinDefensa_ = cocaine_confiscations_MinDefensa.copy()

# Convertir la columna 'fecha_hecho' a tipo datetime (sin especificar formato)
cocaine_confiscations_MinDefensa_['fecha_hecho'] = pd.to_datetime(cocaine_confiscations_MinDefensa_['fecha_hecho'], errors='coerce')

# Extraer solo el año
cocaine_confiscations_MinDefensa_['fecha_hecho'] = cocaine_confiscations_MinDefensa_['fecha_hecho'].dt.year

In [15]:
cocaine_confiscations_MinDefensa_['fecha_hecho'].unique()

array([2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014,
       2013, 2012, 2011, 2010], dtype=int32)

### Verificar valores nulos

In [16]:
cocaine_confiscations_MinDefensa_.isnull().sum()

,0
cod_muni,49
departamento,0
municipio,0
fecha_hecho,0
cantidad,0
unidad,0


Nota: Los nulos de 'cod_muni' los abordamos más adelante

### Estandarización de categorizaciones

La estandarización de categorizaciones es el proceso de uniformizar y normalizar los valores de las categorías en un conjunto de datos para asegurar la consistencia y evitar discrepancias. Esto es crucial para la calidad y precisión de los análisis

In [17]:
# Imprimir categorías únicas para columnas de tipo object
categorical_col = ['departamento', 'municipio', 'unidad']
for column in categorical_col:
    print(f"Categorías en la columna '{column}':")
    print(cocaine_confiscations_MinDefensa_[column].unique())
    print()


Categorías en la columna 'departamento':
['ANTIOQUIA' 'BOGOTÁ D.C.' 'CALDAS' 'CAQUETÁ' 'NARIÑO'
 'NORTE DE SANTANDER' 'SANTANDER' 'VALLE DEL CAUCA' 'ATLÁNTICO' 'BOLÍVAR'
 'CUNDINAMARCA' 'ESPAÑA' 'HUILA' 'PAISES BAJOS' 'QUINDÍO'
 'REPUBLICA DOMINICANA' 'CHOCÓ' 'META' 'PANAMA' 'SAN ANDRÉS ISLAS'
 'AGUAS INTERNACIONALES' 'LA GUAJIRA' 'RISARALDA' 'COSTA RICA' 'HONDURAS'
 'MAGDALENA' 'PUERTO RICO' 'CESAR' 'CÓRDOBA' 'MEXICO' 'ARUBA' 'BELGICA'
 'ECUADOR' 'PUTUMAYO' 'BOYACÁ' 'BRASIL' 'CASANARE' 'TOLIMA' 'CHILE'
 'SUCRE' 'ANTILLAS NEERLANDESAS' 'ARAUCA' 'CAUCA'
 'ESTADOS UNIDOS DE AMERICA' 'FRANCIA' 'EL SALVADOR' 'GUATEMALA'
 'AMAZONAS' 'GUAINÍA' 'PERU' 'PORTUGAL' 'ISLAS VIRGENES BRITANICAS'
 'VAUPÉS' 'INGLATERRA' 'MARTINICA' 'GUAYANA' 'VICHADA' 'BAHAMAS'
 'VENEZUELA' 'GUAVIARE' 'LIBERIA' 'JAMAICA' 'CANADA' 'SURINAM' 'RUSIA'
 'BOLIVAR' 'BOGOTA D.C.' 'CAQUETA' 'QUINDIO' 'ATLANTICO' 'CORDOBA'
 'BOYACA' 'CHOCO' 'SAN ANDRES ISLAS' 'ITALIA' 'TURQUIA' 'REINO UNIDO'
 'CABO VERDE' 'GUAINIA' 'GUINEA' 'V

- Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños

In [18]:
import unicodedata

def remove_accents_and_special_chars(input_str):
    # Normalizar la cadena a NFKD
    nfkd_form = unicodedata.normalize('NFKD', input_str)

    # Eliminar acentos
    no_accents = ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

    # Definir caracteres no deseados
    unwanted_chars = [',', ';', '!', '?', '#', '$', '%']

    # Eliminar caracteres no deseados
    cleaned_str = ''.join([c for c in no_accents if c not in unwanted_chars])

    # Remover espacios en blanco al principio y al final, y convertir a mayúsculas
    result = cleaned_str.strip().upper()

    return result

In [19]:
# Aplicar la función a todas las columnas categóricas
cocaine_confiscations_MinDefensa_[categorical_col] = cocaine_confiscations_MinDefensa_[categorical_col].astype(str)
for col in categorical_col:
    cocaine_confiscations_MinDefensa_[col] = cocaine_confiscations_MinDefensa_[col].apply(remove_accents_and_special_chars)

- Codificación de algunas variables categóricas

### Verificar que los valores en "cod_muni" coincidan con los códigos reales de municipios

Los códigos reales de los municipios de Colombia, están almacenados en la base de datos PostgreSQL del proyecto, en la tabla municipalities dentro del campo dept_mpio_code, junto con la informacion necesaria para georeferenciar todos los municipios y departamentos de Colombia. Este campo guarda el código del municipio en un formato string de exactamente 5 caracteres.

Para hacer esta verificación, previamente exportamos desde la base de datos PostgreSQL un DataFrame con los siguientes campos: dept_name, mpio_name y dept_mpio_code, los cuales contienen la información de los departamentos y municipios oficiales, junto con sus respectivos códigos. Este DataFrame se carga en la siguiente celda y se utiliza para comparar con la columna "codigo_mpio"

In [20]:
dept_mpios_codes = pd.read_csv("/content/drive/MyDrive/analytics_data_proyect/deptos_mupios.csv", index_col=0, dtype={'dept_mpio_code': str})
print(dept_mpios_codes.info())
dept_mpios_codes.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1120
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dept_mpio_code  1121 non-null   object
 1   dept_name       1121 non-null   object
 2   mupio_name      1121 non-null   object
dtypes: object(3)
memory usage: 35.0+ KB
None


,dept_mpio_code,dept_name,mupio_name
0,97001,VAUPES,MITU
1,97161,VAUPES,CARURU
2,97511,VAUPES,PACOA
3,97666,VAUPES,TARAIRA
4,97777,VAUPES,PAPUNAHUA


 -  Verificar la consistencia de la columna "cod_muni" en el df cocaine_confiscations_MinDefensa_

In [21]:
# Asegurarnos de que todos los valores en 'codigo_m' sean strings
cocaine_confiscations_MinDefensa_['cod_muni'] = cocaine_confiscations_MinDefensa_['cod_muni'].astype(str)

# Calcular la longitud de cada valor en la columna
longitudes = cocaine_confiscations_MinDefensa_['cod_muni'].apply(len)

# Verificar si todas las longitudes son iguales
longitudes.nunique() == 1

False

In [22]:
# Mostrar longitudes únicas (opcional)
print(f"Longitudes únicas: {longitudes.unique()}")

Longitudes únicas: [5 7 4 6 2 3 1]


In [23]:
# Contar registros por longitud
long_df = longitudes.value_counts().reset_index()
long_df.rename(columns={'codigo_dane': 'no_dígitos_codigo_dane'}, inplace=True)
long_df['percentage'] = (long_df['count'] / len(longitudes))
long_df.head()

,cod_muni,count,percentage
0,5,194492,0.986898
1,6,1047,0.005313
2,7,964,0.004892
3,4,298,0.001512
4,3,206,0.001045


In [24]:
# Mostrar una muestra de registros para cada longitud
for longitud in longitudes.value_counts().index:
    print(f"Muestra de registros con longitud {longitud}:")
    muestra = cocaine_confiscations_MinDefensa_[longitudes == longitud].head(5)  # Muestra de los primeros 5 registros
    print(muestra[['cod_muni']])
    print()

Muestra de registros con longitud 5:
  cod_muni
0    05040
1    05148
2    05001
3    05001
4    05001

Muestra de registros con longitud 6:
     cod_muni
219    560101
281    560101
285    760101
385    760101
2036   920101

Muestra de registros con longitud 7:
   cod_muni
46  7240101
51  5280101
54  2140101
55  2140101
99  5910101

Muestra de registros con longitud 4:
     cod_muni
108      1111
434      1111
1394     1111
1544     1111
1857     1111

Muestra de registros con longitud 3:
      cod_muni
19339      504
19342      593
19354      502
19356      593
19367      507

Muestra de registros con longitud 2:
      cod_muni
19306       52
19316       52
19368       52
19369       55
19590       52

Muestra de registros con longitud 1:
      cod_muni
24404        7



In [25]:
# Verificar longitudes de 'cod_mupi' igual a 6 dígitos
cod_muni_6 = cocaine_confiscations_MinDefensa_[cocaine_confiscations_MinDefensa_['cod_muni'].apply(len) == 6]
cod_muni_6['departamento'].value_counts()

,count
departamento,
PANAMA,444
COSTA RICA,288
ECUADOR,150
HONDURAS,40
EL SALVADOR,30
ESTADOS UNIDOS,24
ANTILLAS HOLANDESAS,11
BELGICA,11
NICARAGUA,9


In [26]:
# Verificar longitudes de 'cod_mupi' igual a 7 dígitos
cod_muni_7 = cocaine_confiscations_MinDefensa_[cocaine_confiscations_MinDefensa_['cod_muni'].apply(len) == 7]
cod_muni_7['departamento'].value_counts()

,count
departamento,
REPUBLICA DOMINICANA,245
PANAMA,138
GUATEMALA,131
ESPANA,76
ECUADOR,67
MEXICO,48
COSTA RICA,43
PUERTO RICO,29
FRANCIA,27


In [27]:
# Verificar longitudes de 'cod_mupi' igual a 4 dígitos
cod_muni_4 = cocaine_confiscations_MinDefensa_[cocaine_confiscations_MinDefensa_['cod_muni'].apply(len) == 4]
cod_muni_4['departamento'].value_counts()

,count
departamento,
AGUAS INTERNACIONALES,117
REPUBLICA DOMINICANA,99
MAR PACIFICO,54
INGLATERRA,13
FRANCIA,9
VENEZUELA,3
JAMAICA,1
RUMANIA,1
CHINA,1


In [28]:
# Verificar longitudes de 'cod_mupi' igual a 3 dígitos
cod_muni_3 = cocaine_confiscations_MinDefensa_[cocaine_confiscations_MinDefensa_['cod_muni'].apply(len) == 3]
cod_muni_3['departamento'].value_counts()

,count
departamento,
PANAMA,75
SIN ESTABLECER,26
ECUADOR,22
COSTA RICA,21
EL SALVADOR,10
PUERTO RICO,9
ESTADOS UNIDOS,9
HONDURAS,6
GUATEMALA,6


In [29]:
# Verificar longitudes de 'cod_mupi' igual a 2 dígitos
cod_muni_2 = cocaine_confiscations_MinDefensa_[cocaine_confiscations_MinDefensa_['cod_muni'].apply(len) == 2]
cod_muni_2['departamento'].value_counts()

,count
departamento,
MEXICO,31
ESPANA,21
PAISES BAJOS,6
BELGICA,3
BRASIL,2
RUMANIA,2
ITALIA,1


In [30]:
# Verificar longitudes de 'cod_mupi' igual a 1 dígito
cod_muni_1 = cocaine_confiscations_MinDefensa_[cocaine_confiscations_MinDefensa_['cod_muni'].apply(len) == 1]
cod_muni_1['departamento'].value_counts()

,count
departamento,
CUBA,1


Nota: Como se aprecia los 'cod_mun' con longitudes diferentes a 5 dígitos, que en total suman 2.515 registros de un total de 197.074, corresponden a incautaciones hechas en el exterior. Procedemos a excluirlas porque para efectos del proyecto no son importantes

In [31]:
cocaine_confiscations_MinDefensa_ = cocaine_confiscations_MinDefensa_[cocaine_confiscations_MinDefensa_['cod_muni'].apply(len) == 5]

Nota: Con el filtro anterior el 100%  de los codigos que identifican a los municipios en el df cocaine_confiscations_MinDefensa corresponden a string de 5 caracteres, igual a los almacenados en la base de datos del proyecto

- Comparar que los nombres de los departamentos en cada dataframe esten escritos correctamente

Se aplica la función para racionalizar columnas categoricas (Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños) al df 'dept_mpiios_codes'


In [33]:
# Aplicar función a columnas 'dept_name', 'mupio_name'
for col in dept_mpios_codes[['dept_name', 'mupio_name']]:
    dept_mpios_codes[col] = dept_mpios_codes[col].apply(remove_accents_and_special_chars)

In [34]:
# Función para comparar listas y mostrar diferencias
def compare_lists(df1_col, df2_col, label1, label2):
    # Extraer listas únicas y normalizar
    list1 = set(df1_col.str.strip().str.upper().unique())
    list2 = set(df2_col.str.strip().str.upper().unique())

    # Encontrar diferencias
    only_in_list1 = list1 - list2
    only_in_list2 = list2 - list1

    # Imprimir resultados
    print(f"{label1} que no están en {label2}:")
    print(only_in_list1)
    # print(f"{label2} que no están en {label1}:")
    # print(only_in_list2)
    # print()


  - Comparar nombres de departamentos en los dataframes

In [35]:
# Comparar listas de departamento
compare_lists(cocaine_confiscations_MinDefensa_['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en cocaine_confiscations_MinDefensa_", "Departamentos dept_mpios_codes")

Departamentos en cocaine_confiscations_MinDefensa_ que no están en Departamentos dept_mpios_codes:
{'ITALIA', 'VENEZUELA', 'DINAMARCA', 'CURAZAO', 'CROACIA', 'ALEMANIA', 'INGLATERRA', 'SUDAFRICA', 'PERU', 'ANTILLAS HOLANDESAS', 'FRANCIA', 'PORTUGAL', 'ARUBA', 'BELGICA', 'HOLANDA', 'MEXICO', 'ESPANA', 'SAN ANDRES ISLAS', 'BRASIL'}


  Nota: Aún figuran en el df incautaciones hechas en el exterior: 'ITALIA', 'VENEZUELA', 'DINAMARCA', 'CURAZAO', 'CROACIA', 'ALEMANIA', 'INGLATERRA', 'SUDAFRICA', 'PERU', 'ANTILLAS HOLANDESAS', 'FRANCIA', 'PORTUGAL', 'ARUBA', 'BELGICA', 'HOLANDA', 'MEXICO', 'ESPANA', 'BRASIL', además el departamento SAN ANDRES ISLAS está escrito en forma diferente a su nombre real
  
  - Solucionar discrepancias en nombres de departamento SAN ANDRES ISLAS

In [36]:
# Diccionario de mapeo basado en los resultados de la comparación departamentos
depto_mapping = {'SAN ANDRES ISLAS': 'ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA'}

# Reemplazar los nombres incorrectos  el dataframe cocaine_confiscations_MinDefensa_
cocaine_confiscations_MinDefensa_['departamento'] = cocaine_confiscations_MinDefensa_['departamento'].replace(depto_mapping)

 - Confirmar que discrepancia se solucionó

In [37]:
# Comparar listas de departamento
compare_lists(cocaine_confiscations_MinDefensa_['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en cocaine_confiscations_MinDefensa_", "Departamentos dept_mpios_codes")

Departamentos en cocaine_confiscations_MinDefensa_ que no están en Departamentos dept_mpios_codes:
{'ESPANA', 'SUDAFRICA', 'ITALIA', 'BELGICA', 'VENEZUELA', 'PERU', 'DINAMARCA', 'CURAZAO', 'CROACIA', 'ANTILLAS HOLANDESAS', 'HOLANDA', 'BRASIL', 'FRANCIA', 'MEXICO', 'ALEMANIA', 'PORTUGAL', 'ARUBA', 'INGLATERRA'}


In [38]:
# Verificar cuantos registros figuran a nombre de 'VENEZUELA', 'PERU', 'BRASIL', 'ESPANA'
confiscation_overseas = cocaine_confiscations_MinDefensa_[cocaine_confiscations_MinDefensa_['departamento'].isin(['ITALIA', 'VENEZUELA', 'DINAMARCA', 'CURAZAO', 'CROACIA', 'ALEMANIA', 'INGLATERRA', 'SUDAFRICA', 'PERU', 'ANTILLAS HOLANDESAS', 'FRANCIA', 'PORTUGAL', 'ARUBA', 'BELGICA', 'HOLANDA', 'MEXICO', 'ESPANA', 'BRASIL'])]
confiscation_overseas['departamento'].value_counts()

,count
departamento,
MEXICO,79
BELGICA,53
ESPANA,49
HOLANDA,23
VENEZUELA,22
FRANCIA,16
BRASIL,15
INGLATERRA,15
ARUBA,14


In [39]:
# Se procede a eliminar esos registros
cocaine_confiscations_MinDefensa_ = cocaine_confiscations_MinDefensa_[~(cocaine_confiscations_MinDefensa_['departamento'].isin(['ITALIA', 'VENEZUELA', 'DINAMARCA', 'CURAZAO', 'CROACIA', 'ALEMANIA', 'INGLATERRA', 'SUDAFRICA', 'PERU', 'ANTILLAS HOLANDESAS', 'FRANCIA', 'PORTUGAL', 'ARUBA', 'BELGICA', 'HOLANDA', 'MEXICO', 'ESPANA', 'BRASIL']))]

In [40]:
# Comparar nuevamente las listas de departamento
compare_lists(cocaine_confiscations_MinDefensa_['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en cocaine_confiscations_MinDefensa_", "Departamentos dept_mpios_codes")

Departamentos en cocaine_confiscations_MinDefensa_ que no están en Departamentos dept_mpios_codes:
set()


- Verificar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales

In [41]:
# Comparar listas de códigos
compare_lists(cocaine_confiscations_MinDefensa_['cod_muni'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en cocaine_confiscations_MinDefensa_", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en cocaine_confiscations_MinDefensa_ que no están en Códigos de municipios dept_mpios_codes:
set()


### Procesamiento final como preparación para integrarlo a la bd de datos del proyecto

In [42]:
# Eliminar columnas innecesarias
columns_to_drop = ['departamento', 'municipio']
final_cocaine_confiscations_MinDefensa = cocaine_confiscations_MinDefensa_.drop(columns=columns_to_drop)

In [45]:
# Adicionar columna para trazabilidad de la fuente
final_cocaine_confiscations_MinDefensa['source_id'] = 29

In [46]:
final_cocaine_confiscations_MinDefensa.columns

Index(['cod_muni', 'fecha_hecho', 'cantidad', 'unidad', 'source_id'], dtype='object')

In [47]:
# Ajustar nombre de columnas

# Definir el diccionario de traducción
translation_map = {
    'cod_muni': 'dane_code',
    'fecha_hecho': 'year_of_incident',
    'cantidad': 'amount',
    'unidad': 'unit',
    'source_id': 'source_id'
}

# Renombrar las columnas
final_cocaine_confiscations_MinDefensa.rename(columns=translation_map, inplace=True)

In [48]:
#Estructura final del dataset a integrar a la base de datos
final_cocaine_confiscations_MinDefensa.info()

<class 'pandas.core.frame.DataFrame'>
Index: 194164 entries, 0 to 197073
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   dane_code         194164 non-null  object 
 1   year_of_incident  194164 non-null  int32  
 2   amount            194164 non-null  float64
 3   unit              194164 non-null  object 
 4   source_id         194164 non-null  int64  
dtypes: float64(1), int32(1), int64(1), object(2)
memory usage: 8.1+ MB


## Salvar en archivo csv en el drive

In [49]:
# Guardar en archivos CSV en el drive
final_cocaine_confiscations_MinDefensa.to_csv('/content/drive/MyDrive/analytics_data_proyect/initial_transformation/cocaine_confiscations_MinDefensa.csv', index=False)